In [134]:
import requests
import json
import time
import pprint
pp = pprint.PrettyPrinter(indent=4)


In [141]:
# Twitch Client ID 
clientID = 'vb2kmh60pt0tee6o2c11ko6n2t1w9a'

def get_top_100_games(clientID = clientID):
    ''' Given Client ID, pings twitch API for top 100 games. Returns the entire request object'''
    # Need to pass client ID with each request in header
    headers = {'Client-ID': clientID}
    url = '''https://api.twitch.tv/helix/games/top?first=100'''
    r = requests.get(url, headers=headers)
    return r

# r_dict = json.loads(r.text)

In [8]:
r = get_top_100_games()
r_dict = json.loads(r.text)

In [13]:
r_dict['data'][0]['id']

'33214'

In [131]:
# Use this as the time stamp 
time.strftime("%Y-%m-%d %H:%M:%S",time.localtime())

'2019-05-30 19:45:10'

In [157]:
def check_api_limit_reached(req, ignore_limit = False):
    '''Check remaining API pings for request REQ. If API requests is <=1, wait for 30s 
    so for all requests to refill. Returns remaining requests'''
    if int(req.headers['Ratelimit-Remaining']) <= 1: # No more requests, need to pause for 30s
        if ignore_limit:
            return int(req.headers['Ratelimit-Remaining'])
        print('Waiting for API limit to refresh (30s)...')
        time.sleep(30)
        print('Continuing...')
    return int(req.headers['Ratelimit-Remaining'])

In [142]:
def get_top_100_streamers_for_each_game(game_dict):
    '''Given the twitch response for top 100 games, this will cycle through and pull the top 100
    streamers for each game, stored under a dict entry of the title of that game'''
    stream_dict = dict()
    url = 'https://api.twitch.tv/helix/streams?first=100&gameid='
    for game in game_dict['data']:
        req = requests.get(url + game['id'],headers=headers)
        check_api_limit_reached(req)    
        stream_dict[game['name']]=json.loads(req.text)
    return stream_dict

In [ ]:
stream_dict = dict()
follower_dict = dict()
streamer_url = 'https://api.twitch.tv/helix/streams?first=100&gameid=' # + game['id']
follower_url = 'https://api.twitch.tv/helix/users/follows?first=1&to_id='# + game['userid']

for game in r_dict['data']:
    req = requests.get(streamer_url + game['id'],headers=headers)
    check_api_limit_reached(req)    
    
    for stream in json.loads(req.text)['data']:
        follow_req = requests.get(follower_url + stream['user_id'],headers=headers)
        follower_dict[stream['user_id']] = json.loads(follow_req.text)['total']
        check_api_limit_reached(follow_req)    

    stream_dict[game['name']]=json.loads(req.text)


Waiting for API limit to refresh (30s)...
Continuing...
Waiting for API limit to refresh (30s)...


In [176]:
json.loads(req.text)['data'][0]

{'id': '34342686896',
 'user_id': '229729353',
 'user_name': 'CriticalRole',
 'game_id': '509577',
 'community_ids': [],
 'type': 'live',
 'title': 'Critical Role Campaign 2, Episode 65 (5/30 at 7pm PDT) ',
 'viewer_count': 52383,
 'started_at': '2019-05-31T01:39:57Z',
 'language': 'en',
 'thumbnail_url': 'https://static-cdn.jtvnw.net/previews-ttv/live_user_criticalrole-{width}x{height}.jpg',
 'tag_ids': ['d35073b8-0e52-4375-9f75-3dde0c567911',
  'd6bf03b8-42f4-4981-b070-841c7457a9aa',
  '0cb7e377-a308-44e8-91f7-07eea1d96129',
  '6ea6bca4-4712-4ab9-a906-e3336a9d8039']}

In [165]:
r_dict['data'][0]

{'id': '33214',
 'name': 'Fortnite',
 'box_art_url': 'https://static-cdn.jtvnw.net/ttv-boxart/Fortnite-{width}x{height}.jpg'}

In [143]:
stream_dict

NameError: name 'stream_dict' is not defined

In [121]:
with open("data_file.json", "w") as write_file:
    json.dump(stream_dict, write_file)

In [122]:
del stream_dict

In [123]:
with open("data_file.json", "r") as read_file:
    data = json.load(read_file)

In [160]:
data['Fortnite']['data'][0]['user_id']

'229729353'

In [158]:
whos

Variable                              Type             Data/Info
----------------------------------------------------------------
check_api_limit                       function         <function check_api_limit at 0x11eb45b70>
check_api_limit_reached               function         <function check_api_limit_reached at 0x126d38950>
clientID                              str              vb2kmh60pt0tee6o2c11ko6n2t1w9a
create_engine                         function         <function create_engine at 0x11b2c68c8>
data                                  dict             n=99
game                                  dict             n=3
get_top_100_games                     function         <function get_top_100_games at 0x126d277b8>
get_top_100_streamers_for_each_game   function         <function get_top_100_str<...>each_game at 0x126d279d8>
headers                               dict             n=1
json                                  module           <module 'json' from '/ana<...>hon3.7/json/__

In [26]:
 pickle.dump(output_list, open( "output.p", "wb" ) )

In [28]:
!pwd

/Users/jchow/Work/MetisCode/twitch-streamer-recommender/gitRoot/Twitch_Streamers_Recommender/Scraper_Notebook


In [34]:
tfue_id = output_list[0]['data'][0]['user_id']
#[0]['user_id']

'{"total":6166637,"data":[{"from_id":"438578472","from_name":"hentayuu","to_id":"60056333","to_name":"Tfue","followed_at":"2019-05-31T02:15:32Z"}],"pagination":{"cursor":"eyJiIjpudWxsLCJhIjp7IkN1cnNvciI6IjE1NTkyNjg5MzIyMzk5MDQzNTQifX0"}}'

In [153]:
json.loads(testrequest)['total']

6166637

In [90]:
url = 'https://api.twitch.tv/helix/users/follows?first=1&to_id=' + tfue_id
testrequest=requests.get(url,headers=headers).text

In [39]:
# Example api ping through command line
# !curl -H 'Client-ID: vb2kmh60pt0tee6o2c11ko6n2t1w9a' \-X GET https://api.twitch.tv/helix/games/top